# Formating the Output

Combining the JSON formating prompt to the first prompt with slight to none reduction in over output quality.

In [1]:
from reviews import _reviews02

In [2]:
from datetime import datetime
from hashlib import sha224

# create an object with review_ID, review_text and the datetime of creation
def create_review(review_text, review_ID):    
    review_obj = {
        "review_ID": review_ID,
        "review_text": review_text,
        "datetime": datetime.now()
    }

    # create a hash of the review_text for creating unique collection with Chromadb
    hash = sha224(str(review_obj).encode()).hexdigest()
    review_obj['hash'] = hash

    return review_obj

# the temporary review_ID
review_ID = 123
# select a comment to test with
review_text = _reviews02.sample_01
review_obj = create_review(review_text, review_ID)

print(review_obj)
print('\n\n')
print('Len of review_text:', len(review_text.split()))


{'review_ID': 123, 'review_text': "poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should be 60, but no. even with max settings game still looked odd after disabling up - scaling. a $ 70 title should run at 50 - 60 fps on the lowest settings minimum. and trying to optimize the settings to get better frames i have dumped to many hours into it to get a refund. so here i will sit and wait till they fix the performance.", 'datetime': datetime.datetime(2024, 7, 9, 15, 22, 9, 756449), 'hash': 'c177926d0650ff0eadd02401909a3266f9eed4bb0e8ea6823be35a8d'}



Len of review_text: 95


## Attempt 01: Describe the Output

Include a description of the desired JSON format. Test with the 3-3-4 combination

In [3]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.llms import Ollama

llm = Ollama(
    model='llama2:7b-chat-q4_K_M', temperature=0.2, 
    num_gpu = 1,        # disable/enable gpu for testing
    # keep_alive=-1
)
embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")



from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import Chroma

# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=120)
text_spliter = TokenTextSplitter(chunk_size=250, chunk_overlap=40)
docs = text_spliter.create_documents([review_text], metadatas=[{"source":"review_01"}])

# an in-memory Chroma db for one-time usage
# pass a unique collection prevent multiple-thread using the same collection -> retriever collision
temp_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_func,
    collection_name=review_obj['hash'],
)
retriever = temp_db.as_retriever(search_kwargs={"k": 5})

/root/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate
import json

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.
The aspects are {aspects}. For each aspect, output a paragraph with less than 100 words. Then create a JSON with apsects name as key and the paragraph as value. Only output the JSON. Do NOT output other text.

{summaries}

If you don't know the answer, output only \"NA\". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspects of the game from the reviews. Output a paragraph with less than 100 words. The aspects '''

GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']
aspects_response_334 = {k: '' for k in GAME_ASPECTS}

for (start, end) in ((0, 3), (3, 6), (6, 10)):
    aspects = GAME_ASPECTS[start:end]
    my_question = my_question_template + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    print(my_question)

    # retry if response_02 is not a JSON
    for i in range(5):

        chain =  RetrievalQAWithSourcesChain.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={
                "prompt": PromptTemplate(
                    template=prompt_template,
                    input_variables=["summaries", "question"],
                )
            },
            return_source_documents=True,
        )

        response = chain.invoke(
            {
                'question': my_question,
                'aspects': aspects
            }
        )

        print(response)
        print('\n\n')
        print(response['answer'])
        print(response['source_documents'])

        # a small parser to extract the JSON from the response
        # check whether response_02 is a JSON
        # get the first '{'
        first_brace = response['answer'].find('{')
        if first_brace == -1:
            continue
        
        response_ans = response['answer'][first_brace:]

        for i, aspect in enumerate(aspects):
            if f'\"{aspect}\"' not in response_ans:
                print(f'aspect: {aspect} not in response_02. Retry...')
                continue

            if i != len(aspects) - 1:
                next_aspect = aspects[i + 1]
                next_aspect_start = response_ans.find(f'\"{next_aspect}\"')
                if next_aspect_start == -1:
                    print(f'next_aspect: {next_aspect} not in response_02. Retry...')
                    continue
            else:
                next_aspect_start = len(response_ans)

            # check if the value is not in the response
            resp_start = response_ans.find(f'\"{aspect}\"') + len(f'\"{aspect}\"')
            NA_existence = response_ans.find('NA', resp_start + 1, next_aspect_start)
            if NA_existence != -1:
                aspects_response_334[aspect] = 'NA'

            else:
                # get the value, which is wrapped by double quotes after the aspect keyword
                value_start = response_ans.find('\"', resp_start + 1)
                value_end = response_ans.find('\"', value_start + 1)
                aspects_response_334[aspect] = response_ans[value_start + 1:value_end]

        # try:
        #     response_ans_json = json.loads(response_ans)
        # except:
        #     print(f'response_02: \'\'\'{response_ans}\'\'\' is not a JSON. Retry...')
        #     continue

        try:
            for aspect in aspects:
                a = aspects_response_334[aspect]      # attempting to have access to each value
        except:
            print(f'Error in response_02: {aspect}. Retry...')
            continue

        break
    print('\n\n\n')

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


Extract the the following aspects of the game from the reviews. Output a paragraph with less than 100 words. The aspects are : ['Gameplay', 'Narrative', 'Accessibility']


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{'question': "Extract the the following aspects of the game from the reviews. Output a paragraph with less than 100 words. The aspects are : ['Gameplay', 'Narrative', 'Accessibility']", 'aspects': ['Gameplay', 'Narrative', 'Accessibility'], 'answer': '\n{\n"Gameplay": "Poorly optimized with framerate issues.",\n"Narrative": "No mention of narrative quality or lack thereof." ,\n"Accessibility": "NA"\n}', 'sources': '', 'source_documents': [Document(page_content="poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should be 60, but no. even with max settings game still looked odd after disabling up - scaling. a $ 70 title should run at 50 - 60 fps on the lowest settings minimum. and trying to optimize the settings to get better frames i have dumped to many hours into it to get a refund. so here i will sit and wait till they fix the performance.", metadata={'source': 'review_01'})]}




{
"Gameplay": "Poorly optimized

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{'question': "Extract the the following aspects of the game from the reviews. Output a paragraph with less than 100 words. The aspects are : ['Sound', 'Graphics & Art Design', 'Performance']", 'aspects': ['Sound', 'Graphics & Art Design', 'Performance'], 'answer': '\n{\n"Sound": "The sound design is subpar and often drowns out the dialogue.",\n"Graphics & Art Design": "The graphics are poorly optimized and run at 25-35 FPS on both low and ultra settings.",\n"Performance": "Even with max settings, the game still looked odd after disabling upscaling. A $70 title should run at 50-60 FPS on the lowest settings minimum."\n}', 'sources': '', 'source_documents': [Document(page_content="poorly optimized, runs between 25 - 35 fps on both low and ultra settings. you ' d think that if ultra was 30 - 35 then low should be 60, but no. even with max settings game still looked odd after disabling up - scaling. a $ 70 title should run at 50 - 60 fps on the lowest settings minimum. and trying to optimi

---

Trying with different reviews

In [5]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.llms import Ollama

# the temporary review_ID
review_ID = 123
# select a comment to test with
review_text = _reviews02.sample_03
review_obj = create_review(review_text, review_ID)

llm = Ollama(
    model='llama2:7b-chat-q4_K_M', temperature=0.2, 
    num_gpu = 1,        # disable/enable gpu for testing
    # keep_alive=-1
)
embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")



from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores import Chroma

# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=120)
text_spliter = TokenTextSplitter(chunk_size=250, chunk_overlap=40)
docs = text_spliter.create_documents([review_text], metadatas=[{"source":"review_01"}])

# an in-memory Chroma db for one-time usage
# pass a unique collection prevent multiple-thread using the same collection -> retriever collision
temp_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_func,
    collection_name=review_obj['hash'],
)
retriever = temp_db.as_retriever(search_kwargs={"k": 5})

In [6]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate
import json

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.
The aspects are {aspects}. For each aspect, output a paragraph with less than 100 words. Then create a JSON with apsects name as key and the paragraph as value. Only output the JSON. Do NOT output other text.

{summaries}

If you don't know the answer, output only \"NA\". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspects of the game from the reviews. Output a paragraph with less than 100 words. The aspects '''

GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']
aspects_response_334 = {k: '' for k in GAME_ASPECTS}

for (start, end) in ((0, 3), (3, 6), (6, 10)):
    aspects = GAME_ASPECTS[start:end]
    my_question = my_question_template + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    print(my_question)

    # retry if response_02 is not a JSON
    for i in range(5):

        chain =  RetrievalQAWithSourcesChain.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={
                "prompt": PromptTemplate(
                    template=prompt_template,
                    input_variables=["summaries", "question"],
                )
            },
            return_source_documents=True,
        )

        response = chain.invoke(
            {
                'question': my_question,
                'aspects': aspects
            }
        )

        print(response)
        print('\n\n')
        print(response['answer'])
        print(response['source_documents'])

        # check whether response_02 is a JSON
        # get the first '{'
        first_brace = response['answer'].find('{')
        if first_brace == -1:
            continue
        
        response_ans = response['answer'][first_brace:]

        for i, aspect in enumerate(aspects):
            if f'\"{aspect}\"' not in response_ans:
                print(f'aspect: {aspect} not in response_02. Retry...')
                continue

            if i != len(aspects) - 1:
                next_aspect = aspects[i + 1]
                next_aspect_start = response_ans.find(f'\"{next_aspect}\"')
                if next_aspect_start == -1:
                    print(f'next_aspect: {next_aspect} not in response_02. Retry...')
                    continue
            else:
                next_aspect_start = len(response_ans)

            # check if the value is not in the response
            resp_start = response_ans.find(f'\"{aspect}\"') + len(f'\"{aspect}\"')
            NA_existence = response_ans.find('NA', resp_start + 1, next_aspect_start)
            if NA_existence != -1:
                aspects_response_334[aspect] = 'NA'

            else:
                # get the value, which is wrapped by double quotes after the aspect keyword
                value_start = response_ans.find('\"', resp_start + 1)
                value_end = response_ans.find('\"', value_start + 1)
                aspects_response_334[aspect] = response_ans[value_start + 1:value_end]

        # try:
        #     response_ans_json = json.loads(response_ans)
        # except:
        #     print(f'response_02: \'\'\'{response_ans}\'\'\' is not a JSON. Retry...')
        #     continue

        try:
            for aspect in aspects:
                a = aspects_response_334[aspect]      # attempting to have access to each value
        except:
            print(f'Error in response_02: {aspect}. Retry...')
            continue

        break
    print('\n\n\n')

Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Extract the the following aspects of the game from the reviews. Output a paragraph with less than 100 words. The aspects are : ['Gameplay', 'Narrative', 'Accessibility']


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


{'question': "Extract the the following aspects of the game from the reviews. Output a paragraph with less than 100 words. The aspects are : ['Gameplay', 'Narrative', 'Accessibility']", 'aspects': ['Gameplay', 'Narrative', 'Accessibility'], 'answer': 'Here is the JSON output for the requested aspects of the game based on the reviews provided:\n\n{\n"Gameplay": "After you \' ve played for an hour or so and you \' ve learned how the game plays, you can start exploring Paris, which is really cool. Paris is huge and you have a lot of things you can do. Coop missions, random events, and a ton of other cool stuff are some of the things you can keep yourself busy with.",\n"Narrative": "The story isn \' t great either but the latest Assassins Creed games did not have very good stories, so people can \' t blame this game alone.",\n"Accessibility": "You can choose between many different clothings. You can also buy colors so you can make the character dress like a rainbow. There are a lot of diff

Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


{'question': "Extract the the following aspects of the game from the reviews. Output a paragraph with less than 100 words. The aspects are : ['Sound', 'Graphics & Art Design', 'Performance']", 'aspects': ['Sound', 'Graphics & Art Design', 'Performance'], 'answer': 'Here is the JSON output for the requested aspects of the game based on the reviews provided:\n\n{\n"Sound": "The sound design in the game is top-notch, with immersive audio that really adds to the overall experience.",\n"Graphics & Art Design": "The graphics and art design in the game are stunning, with a beautifully rendered Paris that feels alive.",\n"Performance": "While the game does require a pretty good PC to run smoothly, it\'s worth it for the immersive experience."\n}', 'sources': '', 'source_documents': [Document(page_content="okay this game is getting some bad reviews. however i do not think this game is bad. i ' ll agree that you need a pretty good pc to run it. but the game itself isn ' t bad. yes there are some

---

## Attempt 02: Using LangChain JSONOutputParser

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Aspect_2_01(BaseModel):
    gameplay: str = Field(title="Gameplay", description="The mentioned gameplay of the game")
    narrative: str = Field(title="Narrative", description="The mentioned narrative of the game")
    accessibility: str = Field(title="Accessibility", description="The mentioned accessibility of the game")
    sound: str = Field(title="Sound", description="The mentioned sound of the game")
    graphics_art_design: str = Field(title="Graphics & Art Design", description="The mentioned graphics and art design of the game")


class Aspect_2_02(BaseModel):
    performance: str = Field(title="Performance", description="The mentioned performance of the game")
    bug: str = Field(title="Bug", description="The mentioned bugs of the game")
    suggestion: str = Field(title="Suggestion", description="The mentioned suggestion for the game")
    price: str = Field(title="Price", description="The mentioned price of the game")
    overall: str = Field(title="Overall", description="The mentioned overall of the game")

In [9]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import JsonOutputParser

import json

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.
The aspects are {aspects}. For each aspect, output a paragraph with less than 100 words. If the aspect is not mentioned in the reviews, output "NA" for that paragraph. 

{summaries}

Do NOT try to make up an answer. Do NOT output other text.'''

prompt_template_part_2 = \
'''
You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.
The aspects are {aspects}. Output a JSON with the aspects as keys and the paragraphs as values. {format_instructions}. If the aspect is not mentioned in the reviews, output "NA" for that value in the JSON.

The reviews are wrapped by three consecutive single quotes.
\'\'\'
{reviews}
\'\'\'

Do NOT try to make up an answer. Do NOT output other text than the JSON.
'''

my_question_template = \
'''Extract the the following aspects of the game from the reviews. The aspects '''

GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']
aspects_response_2 = {k: '' for k in GAME_ASPECTS}

for (start, end), aspect_obj in zip([(0, 5), (5, 10)], [Aspect_2_01, Aspect_2_02]):
    aspects = GAME_ASPECTS[start:end]
    my_question = my_question_template + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    print(my_question)

    parser = JsonOutputParser(pydantic_object=aspect_obj)

    # retry if response_02 is not a JSON
    for i in range(5):

        relevent_docs = retriever.get_relevant_documents(
            query=my_question,
        )

        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["summaries", "aspects"],
            # partial_variables={"format_instructions": parser.get_format_instructions()},
        )

        chain = prompt | llm

        response_2 = chain.invoke(
            {
                "summaries": '\n'.join([d.page_content for d in relevent_docs]),
                "aspects": aspects
            }
        )

        print(response_2)

        prompt_02 = PromptTemplate(
            template=prompt_template_part_2,
            input_variables=["reviews", "aspects"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )

        chain_02 = prompt_02 | llm

        response_02 = chain_02.invoke(
            {
                "reviews": str(response_2),
                "aspects": aspects
            }
        )

        print(response_2)

        first_brace = response_02.find('{')
        if first_brace == -1:
            continue
        
        response_ans = response_02[first_brace:]

        # in the response, get each key, value pair using str find
        # if the key is not in aspects, retry
        # this is better than trying to convert the response to a JSON and catching the exception
        # as minor errors in the response (like missing ending ',' after each line) can cause the conversion to fail
        # the format of the response is quite stable. It is a JSON with each key, value pair in a new line, with occasionally missing ',' at the end of the line
        for aspect in aspects:
            if f'\"{aspect}\"' not in response_ans:
                print(f'aspect: {aspect} not in response_02. Retry...')
                continue

            # get the value, which is wrapped by double quotes after the aspect keyword
            value_start = response_ans.find('\"', response_ans.find(f'\"{aspect}\"') + len(f'\"{aspect}\"') + 1)
            value_end = response_ans.find('\"', value_start + 1)
            aspects_response_2[aspect] = response_ans[value_start + 1:value_end]

        # try:
        #     response_ans_json = json.loads(response_ans)
        # except:
        #     print(f'response_02: \'\'\'{response_ans}\'\'\' is not a JSON. Retry...')
        #     continue

        try:
            for aspect in aspects:
                a = aspects_response_2[aspect]      # attempting to have access to each value
        except:
            print(f'Error in response_02: {aspect}. Retry...')
            continue
        
        print('\n\n')
        break

    # aspects_response[aspect] = response['answer']
    print('\n\n\n')

Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Extract the the following aspects of the game from the reviews. The aspects are : ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design']

Sure, here are the aspects of the game and their corresponding reviews:

['Gameplay']: "The game itself isn't bad, but you do need a pretty good PC to run it."
["Narrative"]: "The story isn't great either, but the latest Assassin's Creed games didn't have very good stories either."
["Accessibility"]: NA (not mentioned in the reviews)
["Sound"]: NA (not mentioned in the reviews)
["Graphics & Art Design"]: "The graphics are really good."


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2



Sure, here are the aspects of the game and their corresponding reviews:

['Gameplay']: "The game itself isn't bad, but you do need a pretty good PC to run it."
["Narrative"]: "The story isn't great either, but the latest Assassin's Creed games didn't have very good stories either."
["Accessibility"]: NA (not mentioned in the reviews)
["Sound"]: NA (not mentioned in the reviews)
["Graphics & Art Design"]: "The graphics are really good."







Extract the the following aspects of the game from the reviews. The aspects are : ['Performance', 'Bug', 'Suggestion', 'Price', 'Overall']

Here are the aspects of the game and their corresponding reviews:

['Performance']: "However, I do not think this game is bad. Yes, you need a pretty good PC to run it." (NA)

['Bug']: "I have not come across many bugs, but I have experienced a few." (Yes)

['Suggestion']: "After you've played for an hour or so and you've learned how the game plays, you can start exploring Paris, which is really cool." (Yes)


The inserted prompt

In [10]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"performance": {"title": "Performance", "description": "The mentioned performance of the game", "type": "string"}, "bug": {"title": "Bug", "description": "The mentioned bugs of the game", "type": "string"}, "suggestion": {"title": "Suggestion", "description": "The mentioned suggestion for the game", "type": "string"}, "price": {"title": "Price", "description": "The mentioned price of the game", "type": "string"}, "overall": {"title": "Overall", "description": "The mentioned overall of the game", "type": "string"}}, "required": 

It does not work. Yet inspired by the additional content, I decided to create a more concise version of schema for the LLM to understand.

---

## Attempt 03: Provide Custom Schema

Provide a custom schema.

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Extract the following aspect of the game from the reviews.
The aspects are {aspects}. For each aspect, output a paragraph with less than 50 words. Then create a JSON with apsects name as key and the paragraph as value.
{output_format}
Only output the JSON. Do NOT output other text.

{summaries}

If you don't know the answer, output only \"NA\". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspects of the game from the reviews. The aspects '''

GAME_ASPECTS = ['Gameplay', 'Narrative', 'Accessibility', 'Sound', 'Graphics & Art Design', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']
aspects_response_334 = {k: '' for k in GAME_ASPECTS}

for (start, end) in ((0, 3), (3, 6), (6, 10)):
    aspects = GAME_ASPECTS[start:end]

    # formulate the prompts with f-string
    # for RAG
    my_question = my_question_template + f"{'is ' if len(aspects) <= 1 else 'are '}" + ': ' + f'{aspects}'
    print(my_question)

    # for output format
    output_json_template = {k: '...' for k in aspects}
    output_format = "Output the JSON as a single line with no spaces between the key, value pairs. For example, if the aspects are {}, the JSON should be: {}".format(
        str(aspects)[1:-1].replace('\'', '\"'), str(output_json_template).replace('\'', '\"')
    )


    # get the context
    relevant_docs = retriever.get_relevant_documents(query=review_text, k=5)

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["aspects", "output_format", "summaries"]
    )

    chain = prompt | llm

    response_334 = chain.invoke({
        "aspects": str(aspects),
        "summaries": str('\n'.join([d.page_content for d in relevant_docs])),
        "output_format": str(output_format)
    })

    print(response_334)
    print('-'*20)
    
    print('\n\n\n')

Extract the the following aspects of the game from the reviews. The aspects are : ['Gameplay', 'Narrative', 'Accessibility']


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


Here is the JSON output:
{"Gameplay": "...", "Narrative": "...", "Accessibility": "..."}
--------------------




Extract the the following aspects of the game from the reviews. The aspects are : ['Sound', 'Graphics & Art Design', 'Performance']


Number of requested results 5 is greater than number of elements in index 2, updating n_results = 2


{"Sound": "NA", "Graphics & Art Design": "...the latest assassins creed games did not have very good stories...", "Performance": "...you can start exploring paris, which is really cool..."}
--------------------




Extract the the following aspects of the game from the reviews. The aspects are : ['Bug', 'Suggestion', 'Price', 'Overall']
{"Bug": "Some bugs have been experienced, but not many.", "Suggestion": "Explore Paris after learning the gameplay.", "Price": "The price is a bit high, but worth it for the content.", "Overall": "A good game with some minor issues."}
--------------------




